In [1]:
import glob
import json
import os
import random as rnd
import shutil
import sys
from collections import defaultdict

import keras.backend as K
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from IPython.core.interactiveshell import InteractiveShell
from sklearn.metrics.pairwise import euclidean_distances
from tensorflow import keras
from tqdm import tqdm

from src.utils import euclidean_distance, loss

InteractiveShell.ast_node_interactivity = "all"

In [2]:
class CFG:
    def __init__(
        self,
    ):
        self.seed = 39
        self.batch_size = 32
        self.img_size = (512, 81)
        self.n_chanels = 1
        self.n_folds = 6
        self.fold = 0
        self.norm = False
        self.pos_label = 0
        self.n_blocks = 4
        self.emb_len = 1024
        self.kernel_size = (5, 2)
        self.act_fn = "relu"
        self.batch_norm = False
        self.n_epochs = 1000
        self.input_shape = (self.img_size[0], self.img_size[1], self.n_chanels)

In [3]:
cfg = CFG()

In [4]:
root_dir = "/app/_data/artist_data/"
mod_dir = "/app/_data/artist_data/models/test_arch/constr_7/"

In [5]:
test = pd.read_csv(os.path.join(root_dir, "test_meta.tsv"), sep="\t")
test["path"] = test["archive_features_path"].apply(
    lambda x: os.path.join(root_dir, "test_features", x)
)

## TestDataGenerator

In [6]:
class TestDataGenerator(keras.utils.Sequence):
    def __init__(
        self,
        df,
        img_size,
        test=False,
        batch_size=32,
        norm=False,
        n_chanels=1,
    ):
        self.df = df.reset_index(drop=True)
        self.test = test
        self.img_size = img_size
        self.batch_size = batch_size
        self.norm = norm
        self.n_chanels = n_chanels

    def __len__(self):
        return len(self.df) // self.batch_size

    def load_img(self, path):
        img = np.load(path).astype("float32")
        if self.norm:
            img -= img.min()
            img /= img.max()
        if img.shape != self.img_size:
            wpad = self.img_size[1] - img.shape[1]
            wpad_l = wpad // 2
            wpad_r = wpad - wpad_l
            img = np.pad(
                img,
                pad_width=((0, 0), (wpad_l, wpad_r)),
                mode="constant",
                constant_values=0,
            )
        img = np.expand_dims(img, -1)
        if self.n_chanels == 3:
            img = np.concatenate([img, img, img], -1)
        return img

    def _get_one(self, ix):
        img_path, track_id = self.df.loc[ix, ["path", "trackid"]].values
        img = self.load_img(img_path)
        if not self.test:
            artist_id = self.df.loc[ix, "artistid"]
            return {"img": img, "artist_id": artist_id, "track_id": track_id}
        else:
            return {"img": img, "track_id": track_id}

    def __getitem__(self, batch_ix):
        imgs = np.zeros(
            (self.batch_size, self.img_size[0], self.img_size[1], self.n_chanels),
            dtype=np.float32,
        )
        meta = {"track_ids": []}
        if not self.test:
            meta["artist_ids"] = []
        for i in range(self.batch_size):
            data = self._get_one(ix=i + self.batch_size * batch_ix)
            imgs[i] = data["img"]
            meta["track_ids"].append(data["track_id"])
            if not self.test:
                meta["artist_ids"].append(data["artist_id"])

        return {"imgs": imgs, "meta": meta}

## model

In [7]:
input = tf.keras.Input(shape=cfg.input_shape, dtype="float32", name="imgs")
x = keras.layers.Conv2D(
    4,
    cfg.kernel_size,
    activation=cfg.act_fn,
    name="Conv2D_1",
)(input)
x = keras.layers.AveragePooling2D(pool_size=(2, 2), name=f"avg_pool_1")(x)

x = keras.layers.Conv2D(
    filters=16,
    kernel_size=cfg.kernel_size,
    activation=cfg.act_fn,
    name="Conv2D_2",
)(x)
x = keras.layers.AveragePooling2D(pool_size=(2, 2), name=f"avg_pool_2")(x)

x = keras.layers.Conv2D(
    filters=64,
    kernel_size=cfg.kernel_size,
    activation=cfg.act_fn,
    name="Conv2D_3",
)(x)
x = keras.layers.AveragePooling2D(pool_size=(2, 2), name=f"avg_pool_3")(x)

x = keras.layers.Conv2D(
    filters=256,
    kernel_size=cfg.kernel_size,
    activation=cfg.act_fn,
    name="Conv2D_4",
)(x)
x = keras.layers.AveragePooling2D(pool_size=(2, 2), name=f"avg_pool_4")(x)


x = keras.layers.Flatten(name="flatten")(x)
x = keras.layers.Dense(cfg.emb_len, activation=cfg.act_fn, name=f"dense_{cfg.emb_len}")(
    x
)
embedding_net = keras.Model(inputs=input, outputs=x, name=f"embedding")

In [8]:
# emb_len = 2048
# mod_dir = "/app/_data/artist_data/models/test_arch/constr_7/"
# mod = keras.models.load_model(
#     os.path.join(mod_dir, "model.h5"), custom_objects={"contrastive_loss": loss(1)}
# )

In [9]:
# emb_len = 1024
mod_dir = "/app/_data/artist_data/models/test_arch/constr_8/model_879"
mod = keras.models.load_model(mod_dir, custom_objects={"contrastive_loss": loss(1)})

In [10]:
embedding_net.set_weights(mod.weights[:10])

## feature vectors

In [11]:
def find_batch_size(data_size, max_size=550):
    div = [1]
    i = 2
    while i <= data_size:
        if data_size % i == 0:
            div.append(i)
        i += 1
    div = np.array(div)
    max_batch_size = np.max(np.where(div < max_size, div, 0))
    return max_batch_size

In [12]:
gen = TestDataGenerator(
    df=test,
    img_size=cfg.img_size,
    test=True,
    batch_size=find_batch_size(test.shape[0]),
    norm=False,
    n_chanels=1,
)

In [13]:
pred = embedding_net.predict(gen)

/usr/local/lib/python3.8/dist-packages/keras/engine/functional.py:566: UserWarning: Input dict contained keys ['meta'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


161/161 [==============================] - 58s 345ms/step


In [14]:
pred.shape[0] == test.shape[0]

True

In [15]:
def choose_100(
    prediction,
    df,
    val=True,
    path_to_save="/app/_data/artist_data/test_submissions",
    file_ix=1,
):
    dists = euclidean_distances(pred)
    neigh = {}
    with open(os.path.join(path_to_save, f"submission_{file_ix}"), "w") as f:
        for ix in tqdm(range(prediction.shape[0])):
            trackid = df.loc[ix, "trackid"]
            nearest_100 = np.argsort(dists[ix])[:101]
            tracks_100 = df.loc[nearest_100, "trackid"].tolist()
            neigh[trackid] = {"tracks": [x for x in tracks_100 if x != trackid]}
            if val:
                artist_100 = df.loc[nearest_100, "artistid"].tolist()
                neigh[trackid]["artists"] = tracks_100
            f.write(
                "{}\t{}\n".format(
                    trackid,
                    " ".join(list(map(str, tracks_100))),
                )
            )
    return neigh

In [20]:
neigh = choose_100(
    prediction=pred,
    df=test,
    val=False,
    path_to_save=os.path.join(root_dir, "test_submissions"),
    file_ix=2,
)

100% 41377/41377 [02:18<00:00, 299.63it/s]


In [17]:
def load_submission(input_path, max_top_size=100):
    result = {}
    with open(input_path, "r") as finput:
        for line in finput:
            query_trackid, answer_items = line.rstrip().split("\t", 1)
            query_trackid = int(query_trackid)
            ranked_list = []
            for result_trackid in answer_items.split(" "):
                result_trackid = int(result_trackid)
                if result_trackid != query_trackid:
                    ranked_list.append(result_trackid)
                if len(ranked_list) >= max_top_size:
                    break
            result[query_trackid] = ranked_list
    return result

In [21]:
sub = load_submission(
    "/app/_data/artist_data/test_submissions/submission_2", max_top_size=100
)

In [22]:
len(sub) == len(test)

True